In [2]:
!pip install transformers torch nltk rouge

In [3]:
import pandas as pd

# Correct file path
data_path = r'C:\devopsproject\app\amazon.csv'

# Read the CSV
df = pd.read_csv(data_path)

# Show first 5 rows
print(df.head())


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\devopsproject\\app\\amazon.csv'

In [ ]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge

# Initialize the GPT-2 model and tokenizer
model_name = "gpt2"  # Using the base GPT-2 model
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Function to generate text with sampling
def generate_text(prompt, max_length=150):
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)

    # Ensure the length of input is not exceeding the max_length
    input_length = inputs.shape[1]
    max_input_length = 1024  # GPT-2's maximum token length

    if input_length > max_input_length:
        inputs = inputs[:, -max_input_length:]

    outputs = model.generate(
        inputs,
        max_new_tokens=max_length,  # Use max_new_tokens to control number of generated tokens
        num_return_sequences=1,
        do_sample=True,  # Enable sampling
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Reward function using BLEU and ROUGE
def reward_function(generated_text, reference_text):
    bleu_score = sentence_bleu([reference_text.split()], generated_text.split())

    rouge = Rouge()
    rouge_scores = rouge.get_scores(generated_text, reference_text)
    rouge_l_score = rouge_scores[0]['rouge-l']['f']

    # Combine BLEU and ROUGE-L scores (adjust weights as needed)
    combined_score = 0.5 * bleu_score + 0.5 * rouge_l_score

    return combined_score

# Iterative feedback loop with manual feedback input
def iterative_feedback(prompt, reference_text, iterations=5):
    for i in range(iterations):
        # Generate text
        generated_text = generate_text(prompt)
        print(f"Iteration {i + 1}:")
        print(f"Generated Text:\n{generated_text}\n")

        # Evaluate feedback using automatic metrics (BLEU and ROUGE)
        reward = reward_function(generated_text, reference_text)
        print(f"Automatic Reward (BLEU + ROUGE-L): {reward}")

        # Request manual feedback from the user
        manual_score = float(input("Rate the generated text on a scale of 1 to 10 (higher is better): "))
        print(f"Manual Feedback Score: {manual_score}\n")

        # Combine the automatic reward and manual feedback
        combined_reward = (0.7 * manual_score / 10) + (0.3 * reward)  # Adjust the weights as needed
        print(f"Combined Reward (Manual + Automatic): {combined_reward}\n")

        # Simulate updating model based on feedback (in practice, you'd retrain or fine-tune the model)
        print("Adjusting model based on feedback (placeholder)\n")
        print("-" * 50)

# Example usage (using review content as a pseudo-reference)

for index, row in df.iterrows():
    product_name = row['product_name']
    description = row['about_product']
    reference_text = row['review_content']  # Using review as a pseudo-reference for demonstration
    prompt = f"Product Name: {product_name}\nDescription: {description}\nGenerate a compelling product description:"

    print(f"\nProcessing product: {product_name}\n")
    iterative_feedback(prompt, reference_text)
    # Break after the first product for brevity, remove if processing all products is required
    break
